# Real-World Applications: TF-IDF
In this task Hadoop Streaming is used to process Wikipedia articles dump (/data/wiki/en_articles_part).

The purpose of this task is to calculate <code>tf*idf</code> for each pair (word, article) from the Wikipedia dump. Apply the stop words filter to speed up calculations. Term frequency (tf) is a function depending on a term (word) and a document (article):

<code>tf(term, doc_id) = Nt/N</code>,

where Nt - quantity of particular term in the document, N - the total number of terms in the document (without stop words)

Inverse document frequency (idf) is a function depends on a term:

<code>idf(term) = 1/log(1 + Dt)</code>,

where Dt - number of documents in the dataset with the particular term.

You can find more information here: https://en.wikipedia.xn--org/wiki/Tfidf-q82h but use just the formulas mentioned above.

Dataset location: /data/wiki/en_articles_part

Stop words list is in ‘/datasets/stop_words_en.txt’ file.

Format: article_id <i>tab</i> article_text

To parse the articles don’t forget about Unicode (even though this is an English Wikipedia dump, there are many characters from other languages), remove punctuation marks and transform words to lowercase to get the correct quantities. To cope with Unicode we recommend to use the following tokenizer:

Output: <code>tf*idf</code> for term=’labor’ and article_id=12

The result on the sample dataset:

<code>0.000351</code>

<i>Hint: all Wikipedia article_ids are greater than 0. So you can use a dummy article_id=0 to calculate the number of documents with each term.</i>

## Reduce side join solution

In [37]:
%%writefile wiki.dat

1	In this task Hadoop Streaming is used to process Wikipedia articles dump calculations.
2	The purpose of this task is to calculate tf*idf for each pair (word, article) from the Wikipedia dump calculations.
3	Apply the stop words filter to speed up calculations calculations calculations calculations calculations. 
4	Term frequency (tf) is a function depending on a term (word) and a document (article)
5	To parse the articles don’t forget about Unicode (even though this is an English Wikipedia dump, there are many characters from other languages), remove punctuation marks and transform words to lowercase to get the correct quantities.

Writing wiki.dat


In [38]:
%%bash
cp '/datasets/stop_words_en.txt' 'stop_words_en.txt'

### Step 1
First mapper calculates TF for each term. There is no reducer. Output is <code>{term}/t{article_id}/t{tf}</code>

In [3]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

with open('stop_words_en.txt', "r") as f:
    stop_words = set(f.read().splitlines())
    
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    text= re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    text = re.split("\W*\s+\W*", text, flags=re.UNICODE)   
    
    counter = dict()
    for w in text:
        w = w.lower()
        if w in stop_words:
            continue
            
        if w in counter:
            counter[w] += 1
        else:
            counter[w] = 1
    
    total = sum(counter.values())
    
    for term, cnt in counter.iteritems():
        print "%s\t%s\t%f" % (term, article_id, float(cnt)/total) 

Writing mapper1.py


In [39]:
cat wiki.dat | python2 ./mapper1.py | sort | tail

transform	5	0.055556
unicode	5	0.055556
used	1	0.111111
wikipedia	1	0.111111
wikipedia	2	0.100000
wikipedia	5	0.055556
word	2	0.100000
word	4	0.111111
words	3	0.100000
words	5	0.055556


### Step2
Second mapper does nothing and reducer calculates IDF. Input is output of the previous step. Output is <code>{term}/t0/t{idf}</code>.

In [40]:
%%writefile empty_mapper.py

import sys
    
for line in sys.stdin:
    print line

Overwriting empty_mapper.py


In [41]:
cat wiki.dat | python2 ./mapper1.py | python2 ./empty_mapper.py | sort | tail

transform	5	0.055556
unicode	5	0.055556
used	1	0.111111
wikipedia	1	0.111111
wikipedia	2	0.100000
wikipedia	5	0.055556
word	2	0.100000
word	4	0.111111
words	3	0.100000
words	5	0.055556


In [42]:
%%writefile reducer2.py

import sys
from math import log

current_key = None
key_sum = 0
idf = lambda x: 1/log(1+x)

for line in sys.stdin:
    try:
        term, article_id, tf = line.strip().split('\t', 2)
    except ValueError as e:
        continue
        
    if current_key != term:
        if current_key:            
            print "%s\t%d\t%f" % (current_key, 0, idf(key_sum)) # dummy article_id=0   
        current_key = term
        key_sum = 0
    key_sum += 1
    
if current_key:
    print "%s\t%d\t%f" % (current_key, 0, idf(key_sum))

Overwriting reducer2.py


In [43]:
cat wiki.dat | python2 ./mapper1.py | python2 ./empty_mapper.py | sort | python2 ./reducer2.py | tail

task	0	0.910239
term	0	1.442695
tf	0	1.442695
tf*idf	0	1.442695
transform	0	1.442695
unicode	0	1.442695
used	0	1.442695
wikipedia	0	0.721348
word	0	0.910239
words	0	0.910239


### Step 3
Multiple inputs from step1 and step2. Mapper does nothing. Reducer calculates tf*idf. For each term idf comes from the first row (article_id=0).

In [49]:
%%writefile terms.dat

wikipedia	0	0.721348
wikipedia	1	0.111111
wikipedia	2	0.100000
wikipedia	5	0.055556
word	0	0.910239
word	2	0.100000
word	4	0.111111
words	3	0.100000
words	0	0.910239
words	5	0.055556

Overwriting terms.dat


In [65]:
%%writefile reducer3.py

import sys

idf = None

for line in sys.stdin:
    try:
        term, article_id, tf = line.strip().split('\t', 2)
        article_id = int(article_id)
        tf = float(tf)
    except ValueError as e:
        continue
        
    if article_id == 0:
        idf = tf
        continue
    
    print "%s\t%d\t%f" % (term, article_id, tf*idf)

Overwriting reducer3.py


In [66]:
cat terms.dat | python2 ./reducer3.py

wikipedia	1	0.080150
wikipedia	2	0.072135
wikipedia	5	0.040075
word	2	0.091024
word	4	0.101138
words	3	0.091024
words	5	0.050569


In [67]:
%%bash

OUT_STEP1="dir1"
OUT_STEP2="dir2"
OUT_STEP3="dir3"

hdfs dfs -rm -r -skipTrash ${OUT_STEP1} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_STEP2} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_STEP3} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Step 1" \
    -files mapper1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -numReduceTasks 0 \
    -input /data/wiki/en_articles_part \
    -output ${OUT_STEP1} > /dev/null
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Step 2" \
    -D mapreduce.job.reduces=8 \
    -files empty_mapper.py,reducer2.py \
    -mapper "python empty_mapper.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_STEP1} \
    -output ${OUT_STEP2} > /dev/null

# hdfs dfs -cat ${OUT_STEP3}/* |  grep -P 'labor\t12\t' | cut -f3

19/05/01 06:22:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/05/01 06:22:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/05/01 06:22:40 INFO mapred.FileInputFormat: Total input files to process : 1
19/05/01 06:22:40 INFO mapreduce.JobSubmitter: number of splits:2
19/05/01 06:22:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1556667947441_0010
19/05/01 06:22:40 INFO impl.YarnClientImpl: Submitted application application_1556667947441_0010
19/05/01 06:22:40 INFO mapreduce.Job: The url to track the job: http://e08b01388497:8088/proxy/application_1556667947441_0010/
19/05/01 06:22:40 INFO mapreduce.Job: Running job: job_1556667947441_0010
19/05/01 06:22:46 INFO mapreduce.Job: Job job_1556667947441_0010 running in uber mode : false
19/05/01 06:22:46 INFO mapreduce.Job:  map 0% reduce 0%
19/05/01 06:23:02 INFO mapreduce.Job:  map 100% reduce 0%
19/05/01 06:23:03 INFO mapreduce.Job: Job job_1556667947441_0010 completed succ

In [70]:
%%bash

OUT_STEP1="dir1"
OUT_STEP2="dir2"
OUT_STEP3="dir3"

hdfs dfs -rm -r -skipTrash ${OUT_STEP3} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Step 3" \
    -D mapreduce.job.reduces=8 \
    -files empty_mapper.py,reducer3.py \
    -mapper "python empty_mapper.py" \
    -reducer "python reducer3.py" \
    -input ${OUT_STEP1} \
    -input ${OUT_STEP2} \
    -output ${OUT_STEP3} > /dev/null

hdfs dfs -cat ${OUT_STEP3}/* |  grep -P 'labor\t12\t' | cut -f3

rm: `dir3': No such file or directory
19/05/01 06:28:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/05/01 06:28:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/05/01 06:28:39 INFO mapred.FileInputFormat: Total input files to process : 10
19/05/01 06:28:39 INFO mapreduce.JobSubmitter: number of splits:10
19/05/01 06:28:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1556667947441_0012
19/05/01 06:28:40 INFO impl.YarnClientImpl: Submitted application application_1556667947441_0012
19/05/01 06:28:40 INFO mapreduce.Job: The url to track the job: http://e08b01388497:8088/proxy/application_1556667947441_0012/
19/05/01 06:28:40 INFO mapreduce.Job: Running job: job_1556667947441_0012
19/05/01 06:28:46 INFO mapreduce.Job: Job job_1556667947441_0012 running in uber mode : false
19/05/01 06:28:46 INFO mapreduce.Job:  map 0% reduce 0%
19/05/01 06:28:52 INFO mapreduce.Job:  map 40% reduce 0%
19/05/01 06:28:56 INFO mapreduce.Job:  m